In [1]:
import numpy as np
import tensorflow as tf
ks = tf.keras

In [2]:
from kgcnn.data.datasets.ESOLDataset import ESOLDataset
from sklearn.model_selection import train_test_split
dataset = ESOLDataset()
dataset.map_list(method="set_edge_indices_reverse")

ERROR:root:Module 'mol' is deprecated and will be removed in future versions. Please move to 'kgcnn.molecule'.
ERROR:kgcnn.molecule.convert:Can not import `RDKit` package for conversion.
INFO:kgcnn.data.download:Checking and possibly downloading dataset with name ESOL
INFO:kgcnn.data.download:Dataset directory located at C:\Users\patri\.kgcnn\datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.ESOL:Found SDF C:\Users\patri\.kgcnn\datasets\ESOL\delaney-processed.sdf of pre-computed structures.
INFO:kgcnn.data.ESOL:Read molecules from mol-file.
INFO:kgcnn.data.ESOL: ... process molecules 0 from 1128
INFO:kgcnn.molecule.encoder:OneHotEncoder Symbol found ['O', 'C', 'N', 'S', 'Cl', 'P', 'F', 'I', 'Br']
INFO:kgcnn.molecule.encoder:OneHotEncoder Hybridization found [rdkit.Chem.rdchem.HybridizationType.SP3, rdkit.Chem.rdchem.HybridizationType.SP, rdkit.Chem.rdchem.HybridizationType.SP2]
INFO:kgcnn.molecule.encoder:OneH

<ESOLDataset [{'node_symbol': array(['O', 'C', 'C', 'O', 'C', 'O', 'C', 'C', 'O', 'C', 'O', 'C', 'C',
       'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C', 'O', 'C', 'O',
       'C', 'O', 'C', 'O', 'C', 'O'], dtype='<U1'), 'node_number': array([8, 6, 6, 8, 6, 8, 6, 6, 8, 6, 8, 6, 6, 7, 6, 6, 6, 6, 6, 6, 6, 8,
       6, 8, 6, 8, 6, 8, 6, 8, 6, 8]), 'edge_indices': array([[ 0,  1],
       [ 1,  0],
       [ 1,  2],
       [ 2,  1],
       [ 2,  3],
       [ 2, 30],
       [ 3,  2],
       [ 3,  4],
       [ 4,  3],
       [ 4,  5],
       [ 4, 26],
       [ 5,  4],
       [ 5,  6],
       [ 6,  5],
       [ 6,  7],
       [ 7,  6],
       [ 7,  8],
       [ 7, 24],
       [ 8,  7],
       [ 8,  9],
       [ 9,  8],
       [ 9, 10],
       [ 9, 20],
       [10,  9],
       [10, 11],
       [11, 10],
       [11, 12],
       [11, 14],
       [12, 11],
       [12, 13],
       [13, 12],
       [14, 11],
       [14, 15],
       [14, 19],
       [15, 14],
       [15, 16],
       [16, 15],
  

In [3]:
from kgcnn.literature.DMPNN import make_model
from tensorflow.keras.optimizers import Adam

In [4]:
model_config = {
    "name": "DMPNN",
    "inputs": [
        {"shape": [None, 41], "name": "node_attributes", "dtype": "float32", "ragged": True},
        {"shape": [None, 11], "name": "edge_attributes", "dtype": "float32", "ragged": True},
        {"shape": [None, 2], "name": "edge_indices", "dtype": "int64", "ragged": True},
        {"shape": [None, 1], "name": "edge_indices_reverse", "dtype": "int64", "ragged": True}
    ],
    "input_embedding": {"node": {"input_dim": 95, "output_dim": 64},
                        "edge": {"input_dim": 5, "output_dim": 64}},
    "pooling_args": {"pooling_method": "sum"},
    "edge_initialize": {"units": 128, "use_bias": True, "activation": "relu"},
    "edge_dense": {"units": 128, "use_bias": True, "activation": "linear"},
    "edge_activation": {"activation": "relu"},
    "node_dense": {"units": 128, "use_bias": True, "activation": "relu"},
    "verbose": 10, "depth": 5,
    "dropout": {"rate": 0.1},
    "output_embedding": "node",  # For node regression
    "output_to_tensor": False,  # Return ragged tensor output!
    "output_mlp": {
        "use_bias": [True, True, False], "units": [64, 32, 1],
        "activation": ["relu", "relu", "linear"]
    }
}
# Test making model
model = make_model(**model_config)

INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_dim': 5, 'output_dim': 64}, 'graph': {'input_dim': 100, 'output_dim': 64}}, 'pooling_args': {'pooling_method': 'sum'}, 'use_graph_state': False, 'edge_initialize': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'edge_dense': {'units': 128, 'use_bias': True, 'activation': 'linear'}, 'edge_activation': {'activation': 'relu'}, 'node_dense': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'verbose': 10, 'depth': 5, 'dropout': {'rate': 0.1}, 'output_embedding': 'no

In [5]:
dataset.clean(model_config["inputs"])

INFO:kgcnn.data.ESOL:Property 'edge_attributes' is an empty list for graph '934'.
INFO:kgcnn.data.ESOL:Property 'edge_indices' is an empty list for graph '934'.
INFO:kgcnn.data.ESOL:Property 'edge_indices_reverse' is an empty list for graph '934'.


array([934])

In [6]:
for i in range(len(dataset)):
    dataset[i].update({"node_label": np.array(np.expand_dims(dataset[i]["node_number"], axis=-1), dtype="float32")})

In [7]:
train_index, test_index = train_test_split(np.arange(len(dataset)), test_size=0.25, random_state=42)
x_train = dataset[train_index].tensor(model_config["inputs"])
y_train = dataset[train_index].tensor({"name": "node_label", "ragged": True})
x_valid = dataset[test_index].tensor(model_config["inputs"])
y_valid = dataset[test_index].tensor({"name": "node_label", "ragged": True, "dtype": "float32"})
print("inputs:\n", [x.shape for x in x_train])
print("outputs:\n", y_train.shape)

inputs:
 [TensorShape([845, None, 41]), TensorShape([845, None, 11]), TensorShape([845, None, 2]), TensorShape([845, None, 1])]
outputs:
 (845, None, 1)


In [8]:
test = model.predict(x_valid)
print(test.shape)

9/9 [==============================] - 1s 9ms/step
(282, None, 1)


In [9]:
from kgcnn.metrics.loss import RaggedMeanAbsoluteError

In [10]:
model.compile(
    loss=RaggedMeanAbsoluteError(),
    optimizer=Adam(learning_rate=1e-03),
    metrics=["mean_absolute_error"],
)

model.fit(
    x_train, y_train,
    validation_data=(x_valid, y_valid),
    shuffle=True,
    batch_size=32,
    epochs=100,
    verbose=2,
)

Epoch 1/100


C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 12

27/27 - 4s - loss: 2.9411 - mean_absolute_error: 2.9255 - val_loss: 1.9731 - val_mean_absolute_error: 1.9713 - 4s/epoch - 145ms/step
Epoch 2/100
27/27 - 1s - loss: 1.5893 - mean_absolute_error: 1.5857 - val_loss: 1.1098 - val_mean_absolute_error: 1.1057 - 509ms/epoch - 19ms/step
Epoch 3/100
27/27 - 0s - loss: 0.9762 - mean_absolute_error: 0.9769 - val_loss: 0.7553 - val_mean_absolute_error: 0.7527 - 494ms/epoch - 18ms/step
Epoch 4/100
27/27 - 0s - loss: 0.6359 - mean_absolute_error: 0.6344 - val_loss: 0.4433 - val_mean_absolute_error: 0.4411 - 494ms/epoch - 18ms/step
Epoch 5/100
27/27 - 1s - loss: 0.3932 - mean_absolute_error: 0.3931 - val_loss: 0.3502 - val_mean_absolute_error: 0.3480 - 503ms/epoch - 19ms/step
Epoch 6/100
27/27 - 1s - loss: 0.2985 - mean_absolute_error: 0.2951 - val_loss: 0.2536 - val_mean_absolute_error: 0.2513 - 566ms/epoch - 21ms/step
Epoch 7/100
27/27 - 1s - loss: 0.2303 - mean_absolute_error: 0.2307 - val_loss: 0.2119 - val_mean_absolute_error: 0.2097 - 538ms/epo

Epoch 57/100
27/27 - 0s - loss: 0.0386 - mean_absolute_error: 0.0385 - val_loss: 0.0152 - val_mean_absolute_error: 0.0152 - 494ms/epoch - 18ms/step
Epoch 58/100
27/27 - 0s - loss: 0.0321 - mean_absolute_error: 0.0318 - val_loss: 0.0656 - val_mean_absolute_error: 0.0657 - 483ms/epoch - 18ms/step
Epoch 59/100
27/27 - 0s - loss: 0.0308 - mean_absolute_error: 0.0308 - val_loss: 0.0136 - val_mean_absolute_error: 0.0136 - 488ms/epoch - 18ms/step
Epoch 60/100
27/27 - 0s - loss: 0.0398 - mean_absolute_error: 0.0402 - val_loss: 0.0220 - val_mean_absolute_error: 0.0220 - 482ms/epoch - 18ms/step
Epoch 61/100
27/27 - 0s - loss: 0.0526 - mean_absolute_error: 0.0530 - val_loss: 0.0537 - val_mean_absolute_error: 0.0538 - 481ms/epoch - 18ms/step
Epoch 62/100
27/27 - 0s - loss: 0.0358 - mean_absolute_error: 0.0355 - val_loss: 0.0285 - val_mean_absolute_error: 0.0286 - 476ms/epoch - 18ms/step
Epoch 63/100
27/27 - 0s - loss: 0.0287 - mean_absolute_error: 0.0288 - val_loss: 0.0306 - val_mean_absolute_erro

In [11]:
test2 = model.predict(x_valid)

9/9 [==============================] - 1s 8ms/step


In [12]:
test2[0], y_valid[0]

(<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
 array([[5.939304 ],
        [5.9451556],
        [5.984475 ],
        [5.9906645],
        [5.9473505]], dtype=float32)>,
 <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
 array([[6.],
        [6.],
        [6.],
        [6.],
        [6.]], dtype=float32)>)